# GMM + 人工识别，设置虚拟井

In [ ]:
# 确保src目录在Python路径中
import os
import sys

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

sys.path.append(os.path.abspath("../"))

# 导入模块
from src.data_utils import (
    extract_uniform_seismic_samples,
    filter_anomalous_attributes,
    filter_seismic_by_wells,
    identify_attributes,
    parse_petrel_file,
)
from src.feature_selection import select_best_features
from src.visualization import visualize_attribute_map

output_dir = "output"
if not os.path.exists(output_dir):
    os.makedirs(output_dir)


# 设置中文字体
plt.rcParams["font.family"] = "SimHei"  # 黑体 SimHei 支持中文
plt.rcParams["axes.unicode_minus"] = False  # 正常显示负号

## 导入地震数据

In [ ]:
data_seismic_attr = parse_petrel_file("../data/H6-2_attr")

## 导入井点位置

In [ ]:
file_well_position = "../data/well_without_attr.xlsx"
data_well_position = pd.read_excel(file_well_position)

# 选择对应层位的行，丢弃砂厚为 NaN 的行
data_well_purpose_surface = (
    data_well_position[data_well_position["Surface"] == "H6-2"]
    .replace(-999, np.nan)  # 将-999替换为NaN
    .dropna(subset=["Sand Thickness"])  # 丢弃砂厚为NaN的行
    .reset_index(drop=True)  # 重置索引
)
data_well_purpose_surface.head()

## 筛除离群井

In [ ]:
# 筛选离群井
def filter_outlier_wells(well_data, method="iqr", distance_threshold=None):
    """
    筛选并剔除离群井

    参数:
    well_data: 井点数据DataFrame
    method: 'iqr'使用箱线图方法，'distance'使用距离方法
    distance_threshold: 使用distance方法时的距离阈值

    返回:
    filtered_well_data: 过滤后的井点数据
    """
    if method == "iqr":
        # 使用箱线图方法 (IQR = Q3 - Q1)
        Q1_x = well_data["X"].quantile(0.25)
        Q3_x = well_data["X"].quantile(0.75)
        IQR_x = Q3_x - Q1_x

        Q1_y = well_data["Y"].quantile(0.25)
        Q3_y = well_data["Y"].quantile(0.75)
        IQR_y = Q3_y - Q1_y

        # 定义异常值边界 (通常是Q1-1.5*IQR和Q3+1.5*IQR)
        lower_bound_x = Q1_x - 1.5 * IQR_x
        upper_bound_x = Q3_x + 1.5 * IQR_x
        lower_bound_y = Q1_y - 1.5 * IQR_y
        upper_bound_y = Q3_y + 1.5 * IQR_y

        # 筛选正常范围内的井点
        filtered_well_data = well_data[
            (well_data["X"] >= lower_bound_x)
            & (well_data["X"] <= upper_bound_x)
            & (well_data["Y"] >= lower_bound_y)
            & (well_data["Y"] <= upper_bound_y)
        ]

    elif method == "distance":
        if distance_threshold is None:
            raise ValueError("使用distance方法时必须提供distance_threshold参数")

        # 计算井点的中心位置
        center_x = well_data["X"].mean()
        center_y = well_data["Y"].mean()

        # 计算每个井点到中心的距离
        well_data["distance_to_center"] = np.sqrt((well_data["X"] - center_x) ** 2 + (well_data["Y"] - center_y) ** 2)

        # 筛选距离中心不超过阈值的井点
        filtered_well_data = well_data[well_data["distance_to_center"] <= distance_threshold]
        filtered_well_data = filtered_well_data.drop(columns=["distance_to_center"])

    else:
        raise ValueError("method参数必须是'iqr'或'distance'")

    return filtered_well_data.reset_index(drop=True)


# 应用筛选函数
data_well_purpose_surface_filtered = filter_outlier_wells(data_well_purpose_surface, method="iqr")

# 显示筛选前后的井点数量
print(f"筛选前井点数量: {len(data_well_purpose_surface)}")
print(f"筛选后井点数量: {len(data_well_purpose_surface_filtered)}")

# 可视化筛选前后的井点分布
plt.figure(figsize=(12, 6))

plt.subplot(1, 2, 1)
plt.scatter(data_well_purpose_surface["X"], data_well_purpose_surface["Y"], c="blue")
plt.title("筛选前井点分布")
plt.xlabel("X坐标")
plt.ylabel("Y坐标")

plt.subplot(1, 2, 2)
plt.scatter(data_well_purpose_surface_filtered["X"], data_well_purpose_surface_filtered["Y"], c="red")
plt.title("筛选后井点分布")
plt.xlabel("X坐标")
plt.ylabel("Y坐标")

plt.tight_layout()
plt.savefig(os.path.join(output_dir, "well_filtering_comparison.png"))
plt.show()

## test

In [ ]:
def extract_seismic_attributes_at_location(seismic_data, x, y, max_distance=None, num_points=None):
    """
    提取指定位置附近的地震属性平均值

    参数:
    seismic_data: 地震数据，包含X、Y坐标和属性值
    x, y: 目标位置的X、Y坐标
    max_distance: 最大距离范围，超过此距离的点不会被考虑
    num_points: 最多使用的点数量

    返回:
    attributes_dict: 包含平均属性值的字典
    """
    # 计算所有地震点到目标点的距离
    distances = np.sqrt((seismic_data["X"] - x) ** 2 + (seismic_data["Y"] - y) ** 2)

    # 将距离添加到数据中
    temp_data = seismic_data.copy()
    temp_data["distance"] = distances

    # 根据距离排序
    temp_data = temp_data.sort_values("distance")

    # 应用最大距离过滤
    if max_distance is not None:
        temp_data = temp_data[temp_data["distance"] <= max_distance]

    # 应用点数量限制
    if num_points is not None and len(temp_data) > num_points:
        temp_data = temp_data.iloc[:num_points]

    # 如果没有符合条件的点，返回空字典
    if len(temp_data) == 0:
        print(f"警告: 在坐标({x}, {y})附近没有找到符合条件的地震点")
        return {}

    # 获取所有属性列名（排除X、Y和distance列）
    attribute_cols = [col for col in temp_data.columns if col not in ["X", "Y", "distance", "INLINE", "XLINE", "Z"]]

    # 计算每个属性的平均值
    attributes_dict = {}
    for attr in attribute_cols:
        attributes_dict[attr] = temp_data[attr].mean()

    # 返回包含使用的点数量信息
    attributes_dict["num_points_used"] = len(temp_data)
    attributes_dict["avg_distance"] = temp_data["distance"].mean()

    return attributes_dict


# 为所有井点提取地震属性的函数
def extract_seismic_attributes_for_wells(well_data, seismic_data, max_distance=None, num_points=None):
    """
    为所有井点提取地震属性

    参数:
    well_data: 井点数据DataFrame
    seismic_data: 地震数据DataFrame
    max_distance: 最大距离范围
    num_points: 每个井点使用的最多点数量

    返回:
    well_data_with_attributes: 包含地震属性的井点数据
    """
    # 创建结果DataFrame的副本
    well_data_with_attributes = well_data.copy()

    # 遍历每个井点
    for idx, well in well_data.iterrows():
        # 提取该井点处的地震属性
        attributes = extract_seismic_attributes_at_location(
            seismic_data, well["X"], well["Y"], max_distance, num_points
        )

        # 如果找到了属性，则添加到结果中
        if attributes:
            for attr_name, attr_value in attributes.items():
                well_data_with_attributes.loc[idx, attr_name] = attr_value

    return well_data_with_attributes


# 示例：为过滤后的井点提取地震属性
# 使用半径100米范围内的最近10个点
data_well_with_attributes = extract_seismic_attributes_for_wells(
    data_well_purpose_surface_filtered, data_seismic_attr, max_distance=100, num_points=10
)

# 显示结果
print(f"提取属性后的井点数据形状: {data_well_with_attributes.shape}")
print(
    "包含的属性列:",
    [col for col in data_well_with_attributes.columns if col not in data_well_purpose_surface_filtered.columns],
)

# 保存结果
data_well_with_attributes.to_excel(os.path.join(output_dir, "wells_with_seismic_attributes.xlsx"), index=False)